In [1]:
# 自动计算cell的计算时间
%load_ext autotime

time: 295 µs (started: 2021-08-01 22:13:38 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 ch11 #1 change Aug 01, 2021'

git push origin master

In [ ]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[2] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

In [2]:
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 20.5 ms (started: 2021-08-01 22:14:28 +08:00)


# 文本深度学习

> **本章包含**
> * 为机器学习应用程序预处理文本数据
> * 用于文本处理的词袋方法和序列建模方法
> * 变压器架构
> * 序列到序列学习

## 自然语言处理：鸟瞰图